In [1]:
import pandas as pd
import spacy

**Import du fichier train**

In [2]:
df = pd.read_csv(r"C:\Users\ocean\OneDrive\Bureau\Scolarite_EFREI\Cours EFREI\M2\S2 - NLP\sms_dataset.csv")
df.head()


,sms
0,On se voit à 18h devant la gare.
1,"Merci pour ton aide, t’es le meilleur !"
2,"Super soirée hier, merci encore !"
3,Il fait super beau aujourd’hui.
4,"T’es une personne incroyable, je te kiffe !"


**Nettoyage du fichier**

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sms     2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


In [ ]:
# On vérifie si la colonne contient des doublons et on les traite s'il y en a.

filtered_df = df[df['sms'].isna()]
print(filtered_df)

df = df.dropna()

df.info()



Empty DataFrame
Columns: [sms]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sms     2000 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


In [7]:
# On vérifie qu'on a bien que des valeurs avec un type String

print(df['sms'].apply(type).value_counts()) 

sms
<class 'str'>    2000
Name: count, dtype: int64


**Chargement du modèle français de spaCy**

In [21]:
nlp = spacy.load("fr_core_news_sm")  # Chargement du modèle français


In [ ]:
# on applique le modèle spaCy à chaque ligne de la colonne 'text'

df['spacy_doc'] = df['text'].apply(lambda x: nlp(x))

df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),spacy_doc
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"( , I`d, have, responded, ,, if, I, were, going)"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,"( , Sooo, SAD, I, will, miss, you, here, in, S..."
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,"(my, boss, is, bullying, me, ...)"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,"( , what, interview, !, leave, me, alone)"
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"( , Sons, of, *, *, *, *, ,, why, couldn`t, th..."


In [46]:
import re

df["clean_text"] = df["spacy_doc"].apply(lambda doc: " ".join([
    token.lemma_ for token in doc 
    if not token.is_stop and not token.is_punct and not re.search(r"â½", token.text)
]))


In [43]:
df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),spacy_doc,clean_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"( , I`d, have, responded, ,, if, I, were, going)",i`d hav responded if were going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,"( , Sooo, SAD, I, will, miss, you, here, in, S...",sooo SAD will miss you here in san Diego
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,"(my, boss, is, bullying, me, ...)",my boss is bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,"( , what, interview, !, leave, me, alone)",what interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"( , Sons, of, *, *, *, *, ,, why, couldn`t, th...",son of why couldn`t they pouvoir them the re...


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean_text"])

In [48]:
print(vectorizer.get_feature_names_out())  # Liste des mots

['00' '000' '000th' ... 'â½you' 'â½z' 'â½ã']


In [27]:
print(X.toarray())  # Matrice TF-IDF

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [28]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
 
result = classifier("J'adore ce produit, il est excellent !")
print(result)


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.